<a href="https://colab.research.google.com/github/ArchitaKh02/GENERATIVE.AI-EXP/blob/main/GENAI_EXP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q diffusers transformers accelerate safetensors torch --upgrade
!pip install ftfy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 53.3 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import PixArtAlphaPipeline

In [ ]:

# 1) Load a Transformer-based T2I model (PixArt-a / DiT)
model_id = "PixArt-alpha/PixArt-XL-2-512x512"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

pipe = PixArtAlphaPipeline.from_pretrained(
    model_id,
    torch_dtype=dtype,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)

# Optional: enable memory optimizations
if device == "cuda":
    pipe.enable_vae_slicing()
    # Optional, if xformers installed:
    # pipe.enable_xformers_memory_efficient_attention()
else:
    print("Running on CPU – skipping model offload (only needed for GPU).")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at /root/.cache/huggingface/hub/models--PixArt-alpha--PixArt-XL-2-512x512/snapshots/50f702106901db6d0f8b67eb88e814c56ded2692/transformer were not used when initializing PixArtTransformer2DModel: 
 ['caption_projection.y_embedding']


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Running on CPU – skipping model offload (only needed for GPU).


In [ ]:
# 2) Write a good prompt for a face
prompt = (
    "Ultra-realistic close-up portrait of a young adult human, neutral expression, "
    "natural soft lighting, 85mm lens bokeh, high detail skin, photographic quality"
)

negative_prompt = (
    "blurry, low quality, extra fingers, extra eyes, deformed, text, watermark, logo"
)


In [ ]:

# 3) Generate the image
generator = torch.Generator(device=device).manual_seed(42)  # set seed for reproducibility

image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=24,   # increase for slightly higher quality
    guidance_scale=4.5,
    height=512,
    width=512,
    generator=generator
).images[0]

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:

# 4) Save the result
image.save("face.png")
print("Saved to face.png")
# guidance_scale between 3.5 - 6 is typical

Saved to face.png
